# 数据集概览
## Load datasets 

In [1]:
from load import load_all_dataset
X_train, y_train, X_test, y_test = load_all_dataset()
import numpy as np
np.set_printoptions(formatter={"float":lambda x: "{:.3f}".format(x)})

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



## 训练集
### City A (source)

In [2]:
print("source labeled (City A weak & failure):", X_train.source.shape)
# print(np.where(np.isnan(X_train.source[0])))
print("|- labels:", y_train.source.shape)
print(" |- weak=0:", y_train.source[y_train.source==0].shape)
print(" |- failure=1:", y_train.source[y_train.source==1].shape)
print("source background (City A good):", X_train.source_bkg.shape)

source labeled (City A weak & failure): (46110, 672, 10)
|- labels: (46110,)
 |- weak=0: (41341,)
 |- failure=1: (4769,)
source background (City A good): (50862, 672, 10)


### City B (target)

In [14]:
print("target labeled (City B weak & failure):", X_train.target.shape)
print("|- labels:", y_train.target.shape)
print(" |- weak=0:", y_train.target[y_train.target==0].shape)
print(" |- failure=1:", y_train.target[y_train.target==1].shape)
print("target background (City B good):", X_train.target_bkg.shape)

target labeled (City B weak & failure): (438, 672, 10, 1)
|- labels: (438,)
 |- weak=0: (349,)
 |- failure=1: (89,)
target background (City B good): (29592, 672, 10)


## 测试集
### City B (target)

In [4]:
print("target labeled (City B weak & failure):", X_test.target.shape)
print("|- labels:", y_test.target.shape)
print(" |- weak=0:", y_test.target[y_test.target==0].shape)
print(" |- failure=1:", y_test.target[y_test.target==1].shape)
print("target background (City B good):", X_test.target_bkg.shape)

target labeled (City B weak & failure): (17758, 672, 10)
|- labels: (17758,)
 |- weak=0: (15464,)
 |- failure=1: (2294,)
target background (City B good): (47275, 672, 10)


# Residual Neural Network
## 预处理

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
# 去除NaN
from feature_extractor import FeatureExtractor
from numpy import newaxis
class FE(FeatureExtractor):
    def transform(self, X):
        np.nan_to_num(X, copy=False)
        return X[:,:,:,newaxis]

fe = FE()
X_train.source = fe.transform(X_train.source)
X_train.target = fe.transform(X_train.target)
X_test.target = fe.transform(X_test.target)
X_train.source.shape

(46110, 672, 10, 1)

In [7]:
# 将数据集转换为TensorFlow格式
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.source, y_train.source)).batch(16)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_train.target, y_train.target)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.target, y_test.target))

# 额外操作
train_dataset = train_dataset.map( lambda x, y: (tf.image.random_flip_left_right(x), y) )
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

In [8]:
from numpy import zeros, newaxis
a= zeros((4,3))
print(a[0][0])
b=a[:,:,newaxis]
print(b[0][0])

0.0
[0.000]


## 搭建网络模型
参考资料

1. [Introduction to ResNet in TensorFlow 2](https://adventuresinmachinelearning.com/introduction-resnet-tensorflow-2/)

In [9]:
def res_net_block(filters, conv_size, input_data):
    ''' A residual block of 3 layers
    '''
    # 1st layer with batch normalization
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    # 2nd layer with batch normalization, but no activation function
    x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    # 3rd layer is residual addition with an activation function
    x = layers.Add()([x, input_data])
    x = layers.Activation('relu')(x)
    return x

In [10]:
# Layer to be used as an entry point into a Network (a graph of layers).
# https://keras.io/api/layers/core_layers/input/
inputs = keras.Input(shape=(672, 10, 1), name="Input_Layer")
# inputs = layers.InputLayer(input_shape=(672, 10), name="Input_Layer")

# 2D convolution layer (e.g. spatial convolution over images).
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
# filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
# kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.
x = layers.Conv2D(32, 2, activation='relu', name="Layer01")(inputs)
x = layers.Conv2D(64, 2, activation='relu', name="Layer02")(x)

# Max pooling operation for 2D spatial data.
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
# pool_size: integer or tuple of 2 integers, window size over which to take the maximum.
x = layers.MaxPooling2D(2, name="Layer03")(x)

num_res_net_blocks = 10 # 10个ResNet blocks
for i in range(num_res_net_blocks):
    x = res_net_block(64, 2, x)

# [Final layers] a standard CNN layer
x = layers.Conv2D(64, 2, activation='relu', name="Layer-4")(x)
# [Final layers] GAP layer
x = layers.GlobalAveragePooling2D(name="Layer-3")(x)
# [Final layers] dense classification layers
# Just your regular densely-connected NN layer.
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
# units: Positive integer, dimensionality of the output space.
x = layers.Dense(256, activation='relu', name="Layer-2")(x)
# [Final layers] dropout layer
x = layers.Dropout(0.5, name="Layer-1")(x)
# [Final layers] dense classification layers
outputs = layers.Dense(1, activation='softmax', name="Output_Layer")(x)
res_net_model = keras.Model(inputs, outputs)

res_net_model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 672, 10, 1)] 0                                            
__________________________________________________________________________________________________
Layer01 (Conv2D)                (None, 671, 9, 32)   160         Input_Layer[0][0]                
__________________________________________________________________________________________________
Layer02 (Conv2D)                (None, 670, 8, 64)   8256        Layer01[0][0]                    
__________________________________________________________________________________________________
Layer03 (MaxPooling2D)          (None, 335, 4, 64)   0           Layer02[0][0]                    
______________________________________________________________________________________________

## 训练模型

In [11]:
import datetime as dt
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/{}'.format(
      dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S ResNet")), write_images=True),
  ]
res_net_model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['acc'])
res_net_model.fit(train_dataset, epochs=100, steps_per_epoch=200,
          validation_data=valid_dataset,
          validation_steps=3, callbacks=callbacks)

/home/hbai/anaconda3/envs/huawei/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/100
200/200 [==============================] - 43s 202ms/step - loss: 16840.2930 - acc: 0.1081 - val_loss: 430.5132 - val_acc: 0.2917
Epoch 2/100
200/200 [==============================] - 40s 198ms/step - loss: 7787.7861 - acc: 0.0916 - val_loss: 2420.3792 - val_acc: 0.2917
Epoch 3/100
200/200 [==============================] - 40s 198ms/step - loss: 4769.9448 - acc: 0.0797 - val_loss: 243.5034 - val_acc: 0.2917
Epoch 4/100
200/200 [==============================] - 38s 191ms/step - loss: 5470.1958 - acc: 0.0828 - val_loss: 66.2723 - val_acc: 0.2917
Epoch 5/100
200/200 [==============================] - 39s 193ms/step - loss: 6677.0132 - acc: 0.0791 - val_loss: 20372.6543 - val_acc: 0.2917
Epoch 6/100
200/200 [==============================] - 39s 195ms/step - loss: 2620.1509 - acc: 0.0744 - val_loss: 3864.3574 - val_acc: 0.2917
Epoch 7/100
200/200 [==============================] - 39s 196ms/step - loss: 36760.1602 - acc: 0.0781 - val_loss: 10508.8252 - val_acc: 0.2917
Epoch 

## 查看结果

In [12]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.
